<a href="https://colab.research.google.com/github/FerEnnes/FerEnnes/blob/main/MKT_Mini_Agente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
%pip install -q google-generativeai #Serve para instalar a biblioteca oficial do Gemini.


In [31]:
import os, json, math, textwrap #Importa módulos padrão do Python: os (variáveis de ambiente, caminhos),json (serializar/ler JSON),math (funções matemáticas),textwrap (formatar “quebras” de texto para exibir bonito).
import google.generativeai as genai #Importa a SDK do Gemini e dá o apelido genai e é com ela que você configura a API key e chama o modelo.

# 1) lê a chave do painel Secrets (nome EXATO que você criou)
api_key = userdata.get("GOOGLE_APIKEY_ETIC")
assert api_key, "No Secrets: crie 'GOOGLE_APIKEY_ETIC' e conceda acesso ao notebook." #Verifica se a variável de ambiente GOOGLE_API_KEY existe. Se não existir, para a execução e mostra a mensagem

# 2) espelha no os.environ para manter compatível com outros trechos
os.environ["GOOGLE_APIKEY_ETIC"] = api_key

# 3) configura o Gemini
genai.configure(api_key=api_key) #Configura a SDK com a sua API key (lida da variável). Depois disso, qualquer genai.GenerativeModel(...) já usa essa chave.

MODEL = "gemini-2.5-flash"
generation_config = {"temperature": 0.4, "top_p": 0.9, "max_output_tokens": 3000} #: 0 = mais determinístico, alto = mais criativo. Aqui 0.4 (equilíbrio).#“nucleus sampling”; 0.9 limita o vocabulário aos tokens mais prováveis até somar 90% de probabilidade (controle de diversidade). #tamanho máximo da resposta. Evita respostas longas demais.
print("OK: lido dos Secrets e configurado.")


OK: lido dos Secrets e configurado.


In [32]:
#Teste rápido para ver se está funcionando (OK, funcionando., sua chave está 100% conectada):
model = genai.GenerativeModel(MODEL, generation_config=generation_config)
resp = model.generate_content("Diga apenas: OK, funcionando.")
print(resp.text)



OK, funcionando.


In [33]:
#Limpa texto de entrada (sanitize_text): remove quebras de linha e limita o tamanho (até 300 caracteres).
#Valida números (validate_numbers): converte orçamento e CPR para float e bloqueia valores inválidos (não numéricos ou ≤ 0).
#Evita extremos: impõe limites razoáveis (ex.: orçamento > 1e6) para não quebrar a demo e manter o agente seguro.
#CPR (Custo por Resultado) é quanto, em média, se paga por um resultado desejado — ex.: clique, lead ou venda.
def sanitize_text(s: str, max_len=300):
    s = (s or "").strip()
    s = s.replace("\n", " ")
    return s[:max_len]

def validate_numbers(budget, cpr):
    try:
        budget = float(budget)
        cpr = float(cpr)
    except:
        raise ValueError("Orçamento e CPR precisam ser números (ex.: 60 e 3).")
    if budget <= 0 or cpr <= 0:
        raise ValueError("Orçamento e CPR devem ser > 0.")
    if budget > 1e6 or cpr > 1e5:
        raise ValueError("Valores muito altos para a aula. Revise.")
    return budget, cpr


In [34]:
PROMPT_TEMPLATE = """Você é uma assistente de marketing didática.
Tarefa: gerar 3 IDEIAS de post e 2 LEGENDAS curtas por IDEIA, para o tema abaixo.
Políticas (guardrails): não prometa resultados garantidos; seja clara e específica; sem jargões; máximo 140 caracteres por legenda.

Contexto:
- Tema/Nicho: {topic}
- Público: {audience}
- Oferta/Ângulo: {offer}
- Tom de voz: {tone}

Formato de saída (obrigatório):
1) IDEIAS (3 itens numerados, cada ideia com título curto + ângulo)
2) LEGENDAS (para cada ideia, 2 variações curtas com CTA)
Não inclua nada além desses itens.
"""


#Recebe topic, audience, offer e tone, limpa cada um com sanitize_text (guardrail de entrada).
#Preenche o PROMPT_TEMPLATE substituindo os campos {topic}, {audience}, {offer}, {tone} pelos valores limpos.
#Retorna uma única string pronta para enviar ao modelo (o “prompt” final).

def build_prompt(topic, audience, offer, tone):
    return PROMPT_TEMPLATE.format(
        topic=sanitize_text(topic),
        audience=sanitize_text(audience),
        offer=sanitize_text(offer),
        tone=sanitize_text(tone),
    )


# explicando a função da célula 5 - Função do “agente” (gera ideias e legendas) + cálculo em Python

agente_marketing(...)

Orquestra tudo: valida números, monta o prompt, chama o modelo Gemini e calcula a estimativa (orçamento ÷ CPR).
Retorna um dicionário com o texto gerado e os dados da estimativa, já arredondados.

validate_numbers(budget, cpr)

Confere se orçamento e CPR são números válidos (> 0) e dentro de limites razoáveis.
Evita entradas que quebrem a demo ou gerem cálculos sem sentido.

build_prompt(topic, audience, offer, tone)

Limpa os textos (guardrail) e injeta nos campos do template de prompt.
Entrega uma string pronta para o generate_content.

genai.GenerativeModel(model_name=model, generation_config=gen_config)

Cria a “instância” do modelo Gemini que você vai usar.
Aplica o nome do modelo e parâmetros de geração (temperatura, max tokens, etc.).

model.generate_content(prompt)

Envia o prompt ao Gemini e recebe a resposta do modelo.
O texto retornado vai para response.text e é repassado no resultado final.

In [35]:
def agente_marketing(topic, audience, offer, tone, budget, cpr,
                     model=MODEL, gen_config=generation_config):
    budget, cpr = validate_numbers(budget, cpr)
    prompt = build_prompt(topic, audience, offer, tone)

    model = genai.GenerativeModel(model_name=model, generation_config=gen_config)
    response = model.generate_content(prompt)

    # Cálculo feito fora do modelo (controlado)
    estimativa = budget / cpr

    return {
        "saida_texto": response.text,
        "estimativa": {
            "orcamento": budget,
            "cpr": cpr,
            "expressao": f"{budget}/{cpr}",
            "resultado_aproximado": round(estimativa, 2)
        }
    }


In [41]:
# ==== Entrada pelo usuário (com validação) ====

def get_nonempty(prompt_text, max_len=300):
    while True:
        v = input(prompt_text).strip()
        if v:
            # usa seu sanitize_text se já definiu
            return sanitize_text(v, max_len=max_len)
        print("⚠️  Campo obrigatório. Tente novamente.")

def get_float_pos(prompt_text, max_val=None):
    while True:
        v = input(prompt_text).strip().replace(",", ".")
        try:
            x = float(v)
            if x <= 0:
                print("⚠️  Precisa ser > 0.")
                continue
            if max_val and x > max_val:
                print(f"⚠️  Valor muito alto para a aula (>{max_val}). Tente novamente.")
                continue
            return x
        except:
            print("⚠️  Digite um número válido (ex.: 60 ou 3).")

print("Preencha os campos da campanha (use frases curtas):\n")
topic    = get_nonempty("Tema/Nicho: ")
audience = get_nonempty("Público: ")
offer    = get_nonempty("Oferta/Ângulo: ")
tone     = get_nonempty("Tom de voz: ")
budget   = get_float_pos("Orçamento (R$): ", max_val=1e6)
cpr      = get_float_pos("CPR (custo por resultado em R$): ", max_val=1e5)

# Executa o agente
res = agente_marketing(topic, audience, offer, tone, budget, cpr)

# Saída - conteúdo gerado
print("\n=== IDEIAS e LEGENDAS ===\n")
print(res["saida_texto"])

# Saída - estimativa com nome do resultado
resultado_nome = input("\nResultado medido pelo CPR (ex.: cliques, leads, vendas): ").strip() or "resultados"
print("\n=== ESTIMATIVA ===")
print(f"Resultados ≈ orçamento / CPR → {res['estimativa']['expressao']} = {int(res['estimativa']['resultado_aproximado'])} {resultado_nome}")

print("\n(Observação: estimativa é aproximação; não é garantia de performance.)")




Preencha os campos da campanha (use frases curtas):

Tema/Nicho: pizzaria
Público: todos os gêneros de 20 até 35
Oferta/Ângulo: pizza em dobro nas quartas
Tom de voz: imperdível e jovem
Orçamento (R$): 300
CPR (custo por resultado em R$): 0,50

=== IDEIAS e LEGENDAS ===

1) IDEIAS
1.  **Quarta em Dobro:** Destaque a oportunidade de ter duas pizzas pelo preço de uma, ideal para compartilhar ou ter uma extra.
2.  **Quebre a Rotina:** Posicione a oferta como a melhor forma de animar a quarta-feira e sair do tédio da semana.
3.  **Dupla Perfeita:** Enfatize o valor e a conveniência de levar duas pizzas, transformando uma noite comum em algo especial.

2) LEGENDAS
**Para IDEIA 1: Quarta em Dobro**
*   Quarta é dia de alegria em dobro! Peça uma pizza e leve outra grátis. Marque quem vai dividir com você! 🍕🍕
*   Sua quarta merece mais! Leve 2 pizzas pelo preço de 1. Corre pra aproveitar essa oferta imperdível! Peça já! 😉

**Para IDEIA 2: Quebre a Rotina**
*   Cansado da rotina? Quebre a seman